In [55]:
import re
import pandas as pd
from unidecode import unidecode

## Dataset

In [56]:
df = pd.read_csv("/Users/p1con/Documents/UNIVERSIDAD/TFG/datos/dataset.tsv", sep='\t', quoting=3)
df1 = df.loc[df['lang'] == 'es']
barr = df[['id','lang','text']]

In [57]:
barr.head()

,id,lang,text
0,1,es,Toxoplasma gondii es un protozoo intracelular ...
1,2,es,Actualmente los pacientes con cáncer de pulmón...
2,3,es,La asociación de tumor carcinoide y carcinoma ...
3,4,es,La linfadenitis cervical aislada por leishmani...
4,5,es,Introducción. La infección intraabdominal gran...


## Limpiar texto

In [58]:
def clean_text(df, columna):
    df_copy = df.copy()
    df_copy[columna] = df_copy[columna].str.replace('(', '')
    df_copy[columna] = df_copy[columna].str.replace(')', '')
    df_copy[columna] = df_copy[columna].str.replace('AU', '')
    df_copy[columna] = df_copy[columna].str.replace(',', '')
    df_copy[columna] = df_copy[columna].apply(lambda x: unidecode(x))
    return df_copy

In [59]:
barr1 = clean_text(barr, 'text')

## Dataset Abreviaturas

In [60]:
abb_list = pd.read_csv("/Users/p1con/Documents/UNIVERSIDAD/TFG_code/data/abb_list_final.csv", sep=',', quoting=3)
abb_list = abb_list[['Abbreviation','Expansion']]
abb = abb_list.drop_duplicates(subset='Expansion')
abb.count()

Abbreviation    804
Expansion       804
dtype: int64

expansion_counts = abb1.groupby('Abbreviation')['Expansion'].nunique()
filtered_abbreviations = expansion_counts[expansion_counts > 1].index

filtered_df = abb1[abb1['Abbreviation'].isin(filtered_abbreviations)]
abb = filtered_df.drop_duplicates()
abb.count()

In [61]:
abb.groupby('Abbreviation')['Expansion'].nunique()

Abbreviation
5ASA    1
AA      1
AAE     1
AAM     1
AAN     1
       ..
ug      1
ul      1
ung     1
vo      1
vr      1
Name: Expansion, Length: 703, dtype: int64

## Nuevos textos

In [62]:
expansions_set = set(abb['Expansion'])

In [63]:
def contains_expansion(text):
    words = text.split()
    return any(word in expansions_set for word in words)

In [64]:
new_barr = barr1[barr1['text'].apply(contains_expansion)]
new_barr = new_barr[['text']]
new_barr.count()

text    46757
dtype: int64

In [65]:
def replace_first_expansion(row, abbreviations_df, used_expansions):
    text = row['text']
    label = 0
    expansion_column = None
    
    for i, expansion in enumerate(abbreviations_df['Expansion']):
        abbreviation = abbreviations_df['Abbreviation'].iloc[i]
        first_pattern = rf'(?<!\S){re.escape(expansion)}(?!\S).*?(?<!\S){re.escape(abbreviation)}(?!\S)'
        if first_pattern in text and expansion not in used_expansions:
            pattern = rf'(?<!\S){re.escape(expansion)}(?!\S)'
            match = re.search(pattern, text)
            
            if match:
                replacement = match.group()
                text = text.replace(replacement + ' ', '', 1).replace(' ' + replacement, '', 1)
                expansion_column = expansion
                used_expansions.add(expansion)
                label = 1
                break
        
    if label == 0:
        for i, expansion in enumerate(abbreviations_df['Expansion']):
            abbreviation = abbreviations_df['Abbreviation'].iloc[i]
            if expansion in text and abbreviation in text:
                pattern = rf'(?<!\S){re.escape(expansion)}(?!\S)'
                match = re.search(pattern, text)
                abbrev = abbreviations_df[abbreviations_df['Expansion'] == expansion]['Abbreviation'].iloc[0]
                pattern2 = rf'(?<!\S){re.escape(abbrev)}(?!\S)'
                match2 = re.search(pattern2, text)
                
                if match and match2:
                    replacement = match.group()
                    text = text.replace(replacement + ' ', '', 1).replace(' ' + replacement, '', 1)
                    expansion_column = expansion
                    used_expansions.add(expansion)
                    break
    
    row['Exp'] = expansion_column
    row['no_exp'] = text
    return row

In [66]:
def process_text_fields(df, abbreviations_df):
    df_copy = df.copy()
    df_copy = df_copy.apply(lambda row: replace_first_expansion(row, abbreviations_df, set()), axis=1)

    df_copy['no_abb'] = df['text'].copy()
    df_copy['Abb'] = None
    used_expansions = set()
    
    for i, row in df_copy.iterrows():
        expansion = row['Exp']
        if expansion:
            available_abbreviations = abbreviations_df[abbreviations_df['Expansion'] == expansion]['Abbreviation'].tolist()
            for abbreviation in available_abbreviations:
                df_copy.at[i, 'no_abb'] = df_copy.at[i, 'no_abb'].replace(abbreviation, '', 1)
                df_copy.at[i, 'Abb'] = abbreviation
                used_expansions.add(expansion)
                break

    return df_copy

In [67]:
df_modified = process_text_fields(new_barr, abb)
df_modified['text1'] = df_modified['no_exp'].replace(r'\s+', ' ', regex=True)
df_modified['text2'] = df_modified['no_abb'].replace(r'\s+', ' ', regex=True)

In [68]:
df_modified.head()

,text,Exp,no_exp,no_abb,Abb,text1,text2
1,Actualmente los pacientes con cancer de pulmon...,receptor del factor de crecimiento epidermico,Actualmente los pacientes con cancer de pulmon...,Actualmente los pacientes con cancer de pulmon...,EGFR,Actualmente los pacientes con cancer de pulmon...,Actualmente los pacientes con cancer de pulmon...
2,La asociacion de tumor carcinoide y carcinoma ...,None,La asociacion de tumor carcinoide y carcinoma ...,La asociacion de tumor carcinoide y carcinoma ...,None,La asociacion de tumor carcinoide y carcinoma ...,La asociacion de tumor carcinoide y carcinoma ...
5,Introduccion. Las afecciones granulomatosas de...,None,Introduccion. Las afecciones granulomatosas de...,Introduccion. Las afecciones granulomatosas de...,None,Introduccion. Las afecciones granulomatosas de...,Introduccion. Las afecciones granulomatosas de...
9,Introduccion: El tratamiento principal de la g...,None,Introduccion: El tratamiento principal de la g...,Introduccion: El tratamiento principal de la g...,None,Introduccion: El tratamiento principal de la g...,Introduccion: El tratamiento principal de la g...
12,La tina corporal es una infeccion superficial ...,None,La tina corporal es una infeccion superficial ...,La tina corporal es una infeccion superficial ...,None,La tina corporal es una infeccion superficial ...,La tina corporal es una infeccion superficial ...


In [69]:
df_modified.count()

text      46757
Exp        6575
no_exp    46757
no_abb    46757
Abb        6575
text1     46757
text2     46757
dtype: int64

In [70]:
df_no_none = df_modified.dropna(subset=['Abb', 'Exp'])
df_no_none = df_no_none[['text1','text2','text','Exp','Abb']]
df_no_none.reset_index(drop=True, inplace=True)
df_no_none.count()

text1    6575
text2    6575
text     6575
Exp      6575
Abb      6575
dtype: int64

In [71]:
print(df_no_none['Exp'][0])
print(df_no_none['Abb'][0])
print('\n\n')
print(df_no_none['text1'][0])
print('\n\n')
print(df_no_none['text2'][0])
print('\n\n')
print(df_no_none['text'][0])

receptor del factor de crecimiento epidermico
EGFR



Actualmente los pacientes con cancer de pulmon no microcitico CPNM avanzado portadores de mutaciones del EGFR y probablemente en un futuro cercano los pacientes con reordenamientos del gen de la quinasa del linfoma anaplasico ALK pueden recibir un tratamiento especifico basado en el resultado del analisis de biomarcadores. Esto les proporcionara mayor calidad de vida y supervivencia libre de progresion que la quimioterapia convencional. Este documento de consenso nace como una iniciativa conjunta de la Sociedad Espanola de Anatomia Patologica SEAP y de la Sociedad Espanola de Oncologia Medica SEOM y propone recomendaciones diagnosticas y terapeuticas para el paciente con CPNM avanzado basadas en la evidencia cientifica relacionada con el uso de biomarcadores. Por tanto supone una oportunidad para mejorar la eficiencia de la actividad asistencial y la utilizacion de recursos lo que sin duda redundara en un beneficio para estos pacien

In [72]:
def create_text3(row):
    if row['Abb'] in row['text1']:
        return row['text1'].replace(row['Abb'], f"{row['Abb']} ({row['Exp']})", 1)
    else:
        return row['text1']

In [73]:
df_final = df_no_none.copy()
df_final['text3'] = df_final.apply(create_text3, axis=1)
df_final.reset_index(drop=True, inplace=True)
df_final.head()

,text1,text2,text,Exp,Abb,text3
0,Actualmente los pacientes con cancer de pulmon...,Actualmente los pacientes con cancer de pulmon...,Actualmente los pacientes con cancer de pulmon...,receptor del factor de crecimiento epidermico,EGFR,Actualmente los pacientes con cancer de pulmon...
1,OBJETIVO. Valorar las actividades sanitarias e...,OBJETIVO. Valorar las actividades sanitarias e...,OBJETIVO. Valorar las actividades sanitarias e...,presion arterial,PA,OBJETIVO. Valorar las actividades sanitarias e...
2,INTRODUCCION. Valorar modificacion en 182 paci...,INTRODUCCION. Valorar modificacion en 182 paci...,INTRODUCCION. Valorar modificacion en 182 paci...,lipoproteinas de baja densidad,LDL,INTRODUCCION. Valorar modificacion en 182 paci...
3,Presentamos un caso de artritis reactiva secun...,Presentamos un caso de artritis reactiva secun...,Presentamos un caso de artritis reactiva secun...,antiinflamatorios no esteroideos,AINE,Presentamos un caso de artritis reactiva secun...
4,Estudio de dos pacientes con dolor intenso de ...,Estudio de dos pacientes con dolor intenso de ...,Estudio de dos pacientes con dolor intenso de ...,resonancia magnetica nuclear,RMN,Estudio de dos pacientes con dolor intenso de ...


In [74]:
def create_Offset(row):
    first_pos = row['text3'].find(row['Abb'])
    end_pos = first_pos + len(row['Abb'])
    return first_pos, end_pos

In [75]:
df_final2 = df_final.copy()
df_final2[['start_pos', 'end_pos']] = df_final2.apply(create_Offset, axis=1, result_type='expand')
df_final2.reset_index(drop=True, inplace=True)
df_final2.count()

text1        6575
text2        6575
text         6575
Exp          6575
Abb          6575
text3        6575
start_pos    6575
end_pos      6575
dtype: int64

In [76]:
#full_ds = df_final[['Abb','Exp','text1','text3']]
full_ds = df_final2[['text3','Exp','Abb','start_pos','end_pos']]
full_ds.head()

,text3,Exp,Abb,start_pos,end_pos
0,Actualmente los pacientes con cancer de pulmon...,receptor del factor de crecimiento epidermico,EGFR,105,109
1,OBJETIVO. Valorar las actividades sanitarias e...,presion arterial,PA,777,779
2,INTRODUCCION. Valorar modificacion en 182 paci...,lipoproteinas de baja densidad,LDL,551,554
3,Presentamos un caso de artritis reactiva secun...,antiinflamatorios no esteroideos,AINE,437,441
4,Estudio de dos pacientes con dolor intenso de ...,resonancia magnetica nuclear,RMN,278,281


In [77]:
print(full_ds['Exp'][0])
print('\n')
print(full_ds['Abb'][0])
print('\n')
print(full_ds['text3'][0])
print('\n')
print(full_ds['start_pos'][0])

receptor del factor de crecimiento epidermico


EGFR


Actualmente los pacientes con cancer de pulmon no microcitico CPNM avanzado portadores de mutaciones del EGFR (receptor del factor de crecimiento epidermico) y probablemente en un futuro cercano los pacientes con reordenamientos del gen de la quinasa del linfoma anaplasico ALK pueden recibir un tratamiento especifico basado en el resultado del analisis de biomarcadores. Esto les proporcionara mayor calidad de vida y supervivencia libre de progresion que la quimioterapia convencional. Este documento de consenso nace como una iniciativa conjunta de la Sociedad Espanola de Anatomia Patologica SEAP y de la Sociedad Espanola de Oncologia Medica SEOM y propone recomendaciones diagnosticas y terapeuticas para el paciente con CPNM avanzado basadas en la evidencia cientifica relacionada con el uso de biomarcadores. Por tanto supone una oportunidad para mejorar la eficiencia de la actividad asistencial y la utilizacion de recursos lo que sin

In [78]:
#full_ds.to_csv('/Users/p1con/Documents/UNIVERSIDAD/APPLIED MATHEMATICS AND COMPUTING/TFG/datos/dataset-abb-exp-text.csv')
full_ds.count()

text3        6575
Exp          6575
Abb          6575
start_pos    6575
end_pos      6575
dtype: int64

In [79]:
import numpy as np
full_ds.to_csv('/Users/p1con/Documents/UNIVERSIDAD/TFG_code/data/dataset4ft.csv')
#np.savetxt('/Users/p1con/Documents/UNIVERSIDAD/TFG/datos/dataset-bueno.csv', full_ds, delimiter=';')

## Analisis

In [80]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/p1con/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [83]:
def count_words(text):
    words = word_tokenize(text)
    return len(words)

def text_length(text):
    return len(text)

def count_sentences(text):
    sentences = sent_tokenize(text)
    return len(sentences)

In [87]:
texts = full_ds['text3']

full_ds['word_count'] = texts.apply(count_words)
full_ds['text_length'] = texts.apply(text_length)
full_ds['sentence_count'] = texts.apply(count_sentences)

max_word_count = full_ds['word_count'].max()
min_word_count = full_ds['word_count'].min()
max_text_length = full_ds['text_length'].max()
min_text_length = full_ds['text_length'].min()
max_sentence_count = full_ds['sentence_count'].max()
min_sentence_count = full_ds['sentence_count'].min()

avg_word_count = full_ds['word_count'].mean()
avg_text_length = full_ds['text_length'].mean()
avg_sentence_count = full_ds['sentence_count'].mean()

print("Average number of words per text:", avg_word_count)
print("Average text length (in characters):", avg_text_length)
print("Average sentence number:", avg_sentence_count)
print("Maximum number of words in a text:", max_word_count)
print("Minimum number of words in a text:", min_word_count)
print("Maximum text length (in characters):", max_text_length)
print("Minimum text length (in characters):", min_text_length)
print("Maximum sentence count:", max_sentence_count)
print("Minimum sentence count:", min_sentence_count)

Average number of words per text: 270.1029657794677
Average text length (in characters): 1595.3247148288974
Average sentence number: 10.589201520912548
Maximum number of words in a text: 1004
Minimum number of words in a text: 30
Maximum text length (in characters): 5492
Minimum text length (in characters): 171
Maximum sentence count: 64
Minimum sentence count: 1


In [ ]:
df = pd.read_csv("/Users/p1con/Documents/UNIVERSIDAD/TFG/datos/dataset-bueno-offsets.csv", sep=',')
df.count()